In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import numpy as np
import random

In [11]:
def read_excel_file(filename):
    data = pd.read_excel(filename)
    coordinates = data[['X','Y']].values
    customers = data.iloc[1:][['X','Y']].values
    depot = data.iloc[0][['X','Y']].values
    demands = data['Demand'].values
    return coordinates, demands, customers, depot

In [12]:
def calculate_distance(coordinates, i, j):
    x1, y1 = coordinates[i]
    x2, y2 = coordinates[j]
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

In [13]:
def calculate_distance_matrix(coordinates):
    num_points = len(coordinates)
    dist_matrix = np.zeros((num_points, num_points))

    for i in range(num_points):
        for j in range(num_points):
            dist_matrix[i, j] = calculate_distance(coordinates, i, j)

    return dist_matrix

In [14]:
def calculate_total_distance(route, dist_matrix):
    total_distance = 0
    num_points = len(route)

    for i in range(num_points - 1):
        current_node = route[i]
        next_node = route[i + 1]
        total_distance += dist_matrix[current_node, next_node]

    return total_distance

In [15]:
def format_output(routes, coordinates):
    coordinate_routes = []
    for route in routes:
        curr_route = []
        for node_index in route:
            curr_route.append(coordinates[node_index])
        curr_route.append(coordinates[0])  # Adding the depot coordinates at the end
        coordinate_routes.append(curr_route)
    return coordinate_routes

In [ ]:
def nearest_neighbor(dist_matrix, demands, capacity):
    num_points = dist_matrix.shape[0]
    print("Dimension :",num_points)
    visited = np.zeros(num_points, dtype=bool)
    routes = []
    
    headers = ["Current Node", "Nearest Neighbor", "Minimum Distance"]
    while np.sum(visited) < num_points:
        print("\n[ Visited :",np.sum(visited),"\t<","Num Points :",num_points,"]\n")
        data = []
        current_node = 0 
        current_capacity = 0
        route = [current_node]
        visited[current_node] = True  
        
        while current_capacity + demands[current_node] <= capacity:
            current = route[-1]
            nearest = None
            min_dist = float('inf')
            check = 0
            for neighbor in np.where(~visited)[0]:
                if demands[neighbor] + current_capacity <= capacity and dist_matrix[current, neighbor] < min_dist:
                    if(check==0):
                        check = 1;
                    else:
                        nearest = neighbor
                        min_dist = dist_matrix[current, neighbor]

            if nearest is None:
                break
            
            data.append([current,nearest,min_dist])
            route.append(nearest)
            visited[nearest] = True
            current_capacity += demands[nearest]
        
        routes.append(route)
        table = tabulate(data, headers, tablefmt="pretty")
        print(table)
    return routes

In [16]:
import numpy as np
from sklearn.cluster import KMeans
import copy

def GeneratePopulation(dist_matrix, demands, capacity, population_size):
    num_points = dist_matrix.shape[0]
    population = []
    for i in range(population_size):
        clusters  =  random.randint(9,16)
        kmeans = KMeans(n_clusters=clusters, random_state=5).fit(dist_matrix)
        routes = []
        for cluster_label in range(clusters):
            cluster_indices = np.where(kmeans.labels_ == cluster_label)[0]
            if len(cluster_indices) == 0:
                continue

            cluster_demands = demands[cluster_indices]
            total_demand = np.sum(cluster_demands)
            if total_demand > capacity:
                continue

            route = [0]  # Starting point for each route
            current_capacity = 0
            for node_index in cluster_indices:
                if current_capacity + demands[node_index] > capacity:
                    route.append(0)  # Returning to the depot
                    current_capacity = 0
                    route.append(node_index)
                    current_capacity += demands[node_index]
                else:
                    route.append(node_index)
                    current_capacity += demands[node_index]
            route.append(0)  # Returning to the depot at the end
            routes.append(route)

        population.append(copy.deepcopy(routes))  # Deep copy of routes
    return population


In [17]:
def vrp_solver(filename, capacity):
    coordinates, demands, customers, depot = read_excel_file(filename)
    dist_matrix = calculate_distance_matrix(coordinates)
    routes = GeneratePopulation(dist_matrix, demands, capacity, 10)
    formatted_routes = format_output(routes,coordinates)
    return routes

In [18]:
filename = "VRP_Problem1.xlsx"
capacity = 100
solution = vrp_solver(filename, capacity)

In [19]:
print(solution)

[[[0, 7, 0], [0, 11, 18, 23, 28, 31, 0], [0, 15, 0], [0, 4, 12, 0], [0, 1, 14, 19, 21, 0], [0, 9, 17, 0], [0, 10, 0], [0, 32, 0], [0, 3, 16, 0], [0, 0, 2, 22, 0], [0, 6, 24, 0], [0, 29, 0], [0, 13, 0], [0, 20, 0], [0, 5, 25, 27, 30, 0], [0, 8, 26, 0]], [[0, 7, 26, 0], [0, 11, 18, 23, 28, 31, 0], [0, 15, 0], [0, 4, 12, 0], [0, 1, 14, 19, 21, 0], [0, 9, 17, 0], [0, 10, 0], [0, 32, 0], [0, 3, 16, 0], [0, 0, 2, 22, 0], [0, 6, 24, 0], [0, 29, 0], [0, 8, 13, 0], [0, 20, 0], [0, 5, 25, 27, 30, 0]], [[0, 7, 8, 13, 26, 0], [0, 0, 2, 15, 22, 0], [0, 4, 12, 0], [0, 1, 14, 19, 21, 0], [0, 9, 17, 0], [0, 5, 10, 25, 27, 30, 0], [0, 20, 32, 0], [0, 3, 16, 0]], [[0, 7, 8, 13, 26, 0], [0, 0, 2, 15, 22, 0], [0, 4, 12, 0], [0, 1, 14, 19, 21, 0], [0, 9, 17, 0], [0, 5, 10, 25, 27, 30, 0], [0, 20, 32, 0], [0, 3, 16, 0]], [[0, 7, 8, 13, 26, 0], [0, 11, 18, 23, 28, 31, 0], [0, 15, 0], [0, 4, 12, 0], [0, 1, 14, 19, 21, 0], [0, 9, 17, 0], [0, 5, 10, 25, 27, 30, 0], [0, 20, 32, 0], [0, 3, 16, 0], [0, 0, 2, 22, 0

In [20]:
my_list=[]
for routes in solution:
    list=[]
    for route in routes:
        for customer in route:
            if len(list)==0:
                list.append(0)
            elif (len(list)!=0 and list[-1]==0) and customer==0:
                continue
            else:
                list.append(customer)
    
    my_list.append(list)

print(my_list)

[[0, 7, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 10, 0, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0, 29, 0, 13, 0, 20, 0, 5, 25, 27, 30, 0, 8, 26, 0], [0, 7, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 10, 0, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0, 29, 0, 8, 13, 0, 20, 0, 5, 25, 27, 30, 0], [0, 7, 8, 13, 26, 0, 2, 15, 22, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 20, 32, 0, 3, 16, 0], [0, 7, 8, 13, 26, 0, 2, 15, 22, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 20, 32, 0, 3, 16, 0], [0, 7, 8, 13, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 20, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0, 29, 0], [0, 7, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0, 29, 0, 8, 13, 0, 20, 0], [0, 7, 8, 13, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25

In [18]:
import numpy as np
import random

# Problem parameters
num_customers = 32
capacity = 100
num_ants = 10
num_iterations = 100
pheromone_evaporation_rate = 0.25
pheromone_deposit_rate = 1
alpha = 1
beta = 2
Q = 100

# Distance matrix (example, replace with actual distances)
distance_matrix = np.random.randint(1, 100, size=(num_customers + 1, num_customers + 1))
np.fill_diagonal(distance_matrix, 0)

# Demand dictionary
demand = {
    0: 0, 1: 5, 2: 23, 3: 14, 4: 13, 5: 8, 6: 18, 7: 19, 8: 10, 9: 18,
    10: 20, 11: 5, 12: 9, 13: 23, 14: 9, 15: 18, 16: 10, 17: 24, 18: 13,
    19: 14, 20: 8, 21: 10, 22: 19, 23: 14, 24: 13, 25: 14, 26: 2, 27: 23,
    28: 15, 29: 8, 30: 20, 31: 24, 32: 3
}

# Routes
routes = [[0, 7, 8, 13, 26, 0, 2, 15, 22, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 21, 32, 0, 3, 16, 0],
          [0, 7, 8, 13, 26, 0, 11, 8, 23, 28, 29, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 26, 30, 0, 20, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24]]

# Initialize pheromone matrix
pheromone = np.ones((num_customers + 1, num_customers + 1))

def construct_solution(route):
    # Construct solution tour for a single ant based on given route
    ant_solution = [0]  # Start from the depot
    current_capacity = 0
    current_location = 0  # Depot
    visited = set()  # Track visited customers
    for customer in route[1:]:
        if customer == 0:
            ant_solution.append(customer)
            current_capacity = 0
            current_location = customer
        elif current_capacity + demand[customer] <= capacity and customer not in visited:
            ant_solution.append(customer)
            current_capacity += demand[customer]
            current_location = customer
            visited.add(customer)
        else:
            # Return to depot if capacity exceeded or customer already visited
            ant_solution.append(0)
            ant_solution.append(customer)
            current_capacity = demand[customer]
            current_location = customer
            visited.add(customer)
    # Ensure all customers are visited exactly once
    unvisited_customers = set(demand.keys()) - visited
    for customer in unvisited_customers:
        if customer != 0:
            ant_solution.append(customer)
            ant_solution.append(0)
    ant_solution.append(0)  # Return to depot at the end
    return ant_solution






def update_pheromone(pheromone, solutions):
    # Update pheromone levels based on solutions found
    delta_pheromone = np.zeros_like(pheromone)
    for solution in solutions:
        for i in range(len(solution) - 1):
            delta_pheromone[solution[i], solution[i+1]] += Q / len(solution)
    pheromone = (1 - pheromone_evaporation_rate) * pheromone + delta_pheromone
    return pheromone

# Main ACO loop
for iteration in range(num_iterations):
    ant_solutions = []
    for ant in range(num_ants):
        # Choose a route for the ant
        route_index = random.randint(0, 1)
        ant_solution = construct_solution(routes[route_index])
        ant_solutions.append(ant_solution)
    pheromone = update_pheromone(pheromone, ant_solutions)

# Select the best solution found so far as the final route
# This can be based on the total distance or other criteria
best_solution_index = np.argmax([sum(demand[node] for node in solution) for solution in ant_solutions])
best_route = ant_solutions[best_solution_index]

print("Best Route:", best_route)

Best Route: [0, 7, 8, 13, 26, 0, 11, 0, 8, 23, 28, 29, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 0, 26, 30, 0, 20, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 18, 0, 27, 0, 0]


In [17]:
###ACO

In [28]:
import numpy as np
import random

# Problem parameters
num_customers = 32
capacity = 100
num_ants = 10
num_iterations = 100
pheromone_evaporation_rate = 0.25
pheromone_deposit_rate = 1
alpha = 1
beta = 2
Q = 100

# Distance matrix (example, replace with actual distances)
distance_matrix = np.random.randint(1, 100, size=(num_customers + 1, num_customers + 1))
np.fill_diagonal(distance_matrix, 0)

# Demand dictionary
demand = {
    0: 0, 1: 5, 2: 23, 3: 14, 4: 13, 5: 8, 6: 18, 7: 19, 8: 10, 9: 18,
    10: 20, 11: 5, 12: 9, 13: 23, 14: 9, 15: 18, 16: 10, 17: 24, 18: 13,
    19: 14, 20: 8, 21: 10, 22: 19, 23: 14, 24: 13, 25: 14, 26: 2, 27: 23,
    28: 15, 29: 8, 30: 20, 31: 24, 32: 3
}

# Routes
routes = [
[0, 7, 8, 13, 26, 0, 2, 15, 22, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 20, 32, 0, 3, 16, 0], 
[0, 7, 8, 13, 26, 0, 11, 18, 23, 28, 29, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 20, 32, 0, 3, 16, 0, 0, 2, 22, 0, 6, 24, 0],
 [0, 7, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21,0, 9, 17, 0, 10, 0, 32, 0, 3, 16, 0,2, 22, 0, 6, 24, 0, 29, 0, 8, 13, 0, 20, 0, 5, 25, 27, 30, 0],
 [0, 7, 8, 13, 26, 0, 11, 18, 23, 28, 29, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 20, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0], 
[0, 7, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 10, 0, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0, 29, 0, 8, 13, 0, 20, 0, 5, 25, 27, 30, 0], 
[0, 7, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 20, 32, 0, 3, 16,0, 2, 22, 0, 6, 24, 0, 29, 0, 8, 13, 0], 
[0, 7, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 10, 0, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0, 29, 0, 8, 13, 0, 20, 0, 5, 25, 27, 30, 0],
[0, 7, 0 ,11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 10, 0, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0, 29, 0, 13, 0, 20, 0, 5, 25, 27, 30, 0, 8, 26, 0],
[0, 7, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 10, 0, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0, 29, 0, 8, 13, 0, 20, 0, 5, 25, 27, 30, 0],
 [0, 7, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 20, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0, 29, 0, 8, 13, 0]
]

# Initialize pheromone matrix
pheromone = np.ones((num_customers + 1, num_customers + 1))

def construct_solution(route):
    ant_solution = [0]  # Start from the depot
    current_capacity = 0
    current_location = 0  # Depot
    visited = set()  # Track visited customers
    for customer in route[1:]:
        if customer == 0:
            ant_solution.append(customer)
            current_capacity = 0
            current_location = customer
        elif current_capacity + demand[customer] <= capacity and customer not in visited:
            ant_solution.append(customer)
            current_capacity += demand[customer]
            current_location = customer
            visited.add(customer)
        elif customer in visited:
            # Skip the customer if already visited
            continue
        else:
            # Return to depot if capacity exceeded
            ant_solution.append(0)
            ant_solution.append(customer)
            current_capacity = demand[customer]
            current_location = customer
            visited.add(customer)
    # Ensure all customers are visited exactly once
    unvisited_customers = set(demand.keys()) - visited
    for customer in unvisited_customers:
        if customer != 0:
            ant_solution.append(customer)
            ant_solution.append(0)
    ant_solution.append(0)  # Return to depot at the end
    return ant_solution

def update_pheromone(pheromone, solutions):
    delta_pheromone = np.zeros_like(pheromone)
    for solution in solutions:
        for i in range(len(solution) - 1):
            start_node = solution[i]
            end_node = solution[i+1]
            delta_pheromone[start_node, end_node] += Q / distance_matrix[start_node, end_node]
    pheromone = (1 - pheromone_evaporation_rate) * pheromone + delta_pheromone
    return pheromone

# Main ACO loop
for iteration in range(num_iterations):
    ant_solutions = []
    for ant in range(num_ants):
        # Choose a route for the ant
        route_index = random.randint(0, 1)
        ant_solution = construct_solution(routes[route_index])
        ant_solutions.append(ant_solution)
    pheromone = update_pheromone(pheromone, ant_solutions)

# Select the best solution found so far as the final route
# This can be based on the total distance or other criteria
best_solution_index = np.argmax([sum(demand[node] for node in solution) for solution in ant_solutions])
best_route = ant_solutions[best_solution_index]

print("Best Route:", best_route)

Best Route: [0, 7, 8, 13, 26, 0, 11, 18, 23, 28, 29, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 20, 32, 0, 3, 16, 0, 0, 2, 22, 0, 6, 24, 0, 0]


C:\Users\vuppu\AppData\Local\Temp\ipykernel_16608\2518234958.py:84: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pheromone[start_node, end_node] += Q / distance_matrix[start_node, end_node]


In [ ]:
[ [0, 7, 8, 13, 26, 0, 2, 15, 22, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 21, 32, 0, 3, 16, 0],
    [0,7,26,0,11,18,23,28,31,0,15,0,4,12,0,1,14,0,19,21,0,9,17,0,10,32,0,3,16,2,22,0,6,24,0,29,0,8,13,0,20,0,5,25,27,30,0],
          [0,7,8,13,26,0,11,18,23,28,29,31,0,4,12,0,1,14,19,21,0,9,17,0,5,10,25,27,30,0,20,32,0,3,16,0,2,22,0,6,24,0],
         [0, 7, 8, 13, 26, 0, 11, 8, 23, 28, 29, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 26, 30, 0, 20, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24]]

In [3]:
##PSO


In [4]:
import numpy as np

class Particle:
    def __init__(self, num_customers, capacity, distance_matrix, demand):
        self.num_customers = num_customers
        self.capacity = capacity
        self.distance_matrix = distance_matrix
        self.demand = demand
        self.position = np.zeros(num_customers + 2, dtype=int)  # Include two zeros for start and end
        self.velocity = np.zeros(num_customers + 2, dtype=int)  # Include two zeros for start and end
        self.best_position = None
        self.best_fitness = float('inf')
    
    def initialize(self):
        self.position[1:-1] = np.random.permutation(range(1, self.num_customers + 1))
        self.velocity[1:-1] = np.random.permutation(range(1, self.num_customers + 1))
        self.position[-1] = self.position[0]  # Set the last element to 0 for end
        self.velocity[-1] = self.velocity[0]  # Set the last element to 0 for end
        self.best_position = self.position.copy()
        self.best_fitness = self.fitness(self.position)
    
    def update_velocity(self, global_best_position, c1, c2, w):
        r1 = np.random.rand(self.num_customers + 2)
        r2 = np.random.rand(self.num_customers + 2)
        self.velocity = (w * self.velocity) + (c1 * r1 * (self.best_position - self.position)) + \
                        (c2 * r2 * (global_best_position - self.position))
    
    def update_position(self):
        self.position = np.argsort(self.position + self.velocity)
        self.position[-1] = self.position[0]  # Ensure the last element is 0 for end
    
    def fitness(self, position):
        fitness = 0
        current_capacity = self.capacity
        current_route = []
        
        for customer in position:
            if customer == 0:
                if current_route:
                    fitness += self.distance_matrix[current_route[-1], 0]
                current_route = []
                current_capacity = self.capacity
            else:
                demand = self.demand[customer]
                if demand <= current_capacity:
                    current_capacity -= demand
                else:
                    fitness = float('inf')  # Violation of capacity constraint
                    break
                current_route.append(customer)
        
        if current_route:
            fitness += self.distance_matrix[current_route[-1], 0]
        
        return fitness



def particle_swarm_optimization(num_customers, capacity, num_particles, num_iterations, c1, c2, w,
                                distance_matrix, demand, routes):
    particles = []
    global_best_position = None
    global_best_fitness = float('inf')
    
    # Initialize global_best_position with the position of the first particle
    # before entering the optimization loop
    first_particle = Particle(num_customers, capacity, distance_matrix, demand)
    first_particle.initialize()
    global_best_position = first_particle.best_position
    global_best_fitness = first_particle.best_fitness
    
    for _ in range(num_particles):
        particle = Particle(num_customers, capacity, distance_matrix, demand)
        particle.initialize()
        particles.append(particle)
        
    for _ in range(num_iterations):
        for particle in particles:
            particle.update_velocity(global_best_position, c1, c2, w)
            particle.update_position()
            
            current_fitness = particle.fitness(particle.position)
            if current_fitness < particle.best_fitness:
                particle.best_position = particle.position.copy()
                particle.best_fitness = current_fitness
            
            if current_fitness < global_best_fitness:
                global_best_position = particle.position.copy()
                global_best_fitness = current_fitness
                
    # Find the best route from the provided routes list
    best_route_fitness = float('inf')
    best_route = None
    for route in routes:
        route_fitness = first_particle.fitness(route)
        if route_fitness < best_route_fitness:
            best_route_fitness = route_fitness
            best_route = route
    
    return best_route, best_route_fitness




# Example usage
num_customers = 32
capacity = 100
num_particles = 10
num_iterations = 100
c1 = 2  # Cognitive parameter
c2 = 2  # Social parameter
w = 0.5  # Inertia weight

# Distance matrix (example, replace with actual distances)
# Distance matrix (example, replace with actual distances)
distance_matrix = np.random.randint(1, 100, size=(num_customers + 2, num_customers + 2))  # Adjust size to (num_customers + 2, num_customers + 2)
np.fill_diagonal(distance_matrix, 0)



# Demand dictionary
demand = {
    0: 0, 1: 5, 2: 23, 3: 14, 4: 13, 5: 8, 6: 18, 7: 19, 8: 10, 9: 18,
    10: 20, 11: 5, 12: 9, 13: 23, 14: 9, 15: 18, 16: 10, 17: 24, 18: 13,
    19: 14, 20: 8, 21: 10, 22: 19, 23: 14, 24: 13, 25: 14, 26: 2, 27: 23,
    28: 15, 29: 8, 30: 20, 31: 24, 32: 3,33 : 10
}


routes = [
[0, 7, 8, 13, 26, 0, 2, 15, 22, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 20, 32, 0, 3, 16, 0], 
[0, 7, 8, 13, 26, 0, 11, 18, 23, 28, 29, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 20, 32, 0, 3, 16, 0, 0, 2, 22, 0, 6, 24, 0],
 [0, 7, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21,0, 9, 17, 0, 10, 0, 32, 0, 3, 16, 0,2, 22, 0, 6, 24, 0, 29, 0, 8, 13, 0, 20, 0, 5, 25, 27, 30, 0],
 [0, 7, 8, 13, 26, 0, 11, 18, 23, 28, 29, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 20, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0], 
[0, 7, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 10, 0, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0, 29, 0, 8, 13, 0, 20, 0, 5, 25, 27, 30, 0], 
[0, 7, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 20, 32, 0, 3, 16,0, 2, 22, 0, 6, 24, 0, 29, 0, 8, 13, 0], 
[0, 7, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 10, 0, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0, 29, 0, 8, 13, 0, 20, 0, 5, 25, 27, 30, 0],
[0, 7, 0 ,11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 10, 0, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0, 29, 0, 13, 0, 20, 0, 5, 25, 27, 30, 0, 8, 26, 0],
[0, 7, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 10, 0, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0, 29, 0, 8, 13, 0, 20, 0, 5, 25, 27, 30, 0],
 [0, 7, 26, 0, 11, 18, 23, 28, 31, 0, 15, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 20, 32, 0, 3, 16, 0, 2, 22, 0, 6, 24, 0, 29, 0, 8, 13, 0]
]


best_position, best_fitness = particle_swarm_optimization(num_customers, capacity, num_particles,
                                                          num_iterations, c1, c2, w,
                                                          distance_matrix, demand, routes)

print("Best route:",best_position)
print("Best fitness:", best_fitness)

Best route: [0, 7, 8, 13, 26, 0, 2, 15, 22, 0, 4, 12, 0, 1, 14, 19, 21, 0, 9, 17, 0, 5, 10, 25, 27, 30, 0, 20, 32, 0, 3, 16, 0]
Best fitness: 270
